In [60]:
import os
import warnings
import pandas as pd
from openpyxl import load_workbook, Workbook
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
from copy import copy
import requests
from bs4 import BeautifulSoup
import wget
import ssl
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

data_atual = datetime.now()
data_atual_f = data_atual.date().strftime('%d-%m-%Y')
data_ontem = data_atual - timedelta(days=1)
data_ontem_f = data_ontem.date().strftime('%d-%m-%Y')

ssl._create_default_https_context = ssl._create_unverified_context

try:
    url = f'https://extrator.sefaz.al.gov.br/DESPESAS/COMPARATIVO-DESPESAS/despesa_empenhado_liquidado_pago_2024_siafe_gerado_em_{data_ontem_f}.xlsx'
    arquivo = os.path.join('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/DASHBOARDS POWERBI/Base python', f'despesa_empenhado_liquidado_pago_2024_siafe_gerado_em_{data_ontem_f}.xlsx')
    arquivo_csv = os.path.join('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/DASHBOARDS POWERBI/Base python', f'despesa_empenhado_liquidado_pago_2024_siafe_gerado_em_{data_ontem_f}.csv')
    wget.download(url, arquivo)
except:
    url = f'https://extrator.sefaz.al.gov.br/DESPESAS/COMPARATIVO-DESPESAS/despesa_empenhado_liquidado_pago_2024_siafe_gerado_em_{data_atual_f}.xlsx'
    arquivo = os.path.join('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/DASHBOARDS POWERBI/Base python', f'despesa_empenhado_liquidado_pago_2024_siafe_gerado_em_{data_atual_f}.xlsx')
    arquivo_csv = os.path.join('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/DASHBOARDS POWERBI/Base python', f'despesa_empenhado_liquidado_pago_2024_siafe_gerado_em_{data_atual_f}.csv')
    wget.download(url, arquivo)

base_arquivo = os.path.join('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/SCRIPTS/Bases empenhado - Layne', f'Base Acompanhamento - atualizado {data_atual_f}.xlsx')


# Baixar o arquivo
wget.download(url, arquivo)
df = pd.read_excel(arquivo)

df_f = df[['DESCRICAO_UO', 'PT', 'PT_DESCRICAO', 'NATUREZA6', 'VALOR_EMPENHADO', 'MES']]
for i in ['DESCRICAO_UO', 'PT', 'PT_DESCRICAO', 'NATUREZA6', 'MES']:
    df_f[i] = df_f[i].astype(str)

# concatenando as colunas para o merge
df_x = df_f
df_x['concat'] = df_x['DESCRICAO_UO']
for i in ['PT', 'NATUREZA6']:
    df_x['concat'] = df_x['concat'] + df_x[i]

# base com as informacoes
info = pd.read_excel('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/DASHBOARDS POWERBI/Base python/Base acompanhamento Layne.xlsx')

# fazendo o merge para cada valor e cada mês

for i in df_f['MES'].unique():
    info[i] = ''
    teste = df_x.loc[df_x['MES'] == i]
    for j in info['concat'].values:
     info.loc[info.concat == j, i] = teste.loc[teste.concat == j]['VALOR_EMPENHADO'].sum()

# Dropando o concat

info.drop(columns='concat', inplace=True)

# Criando a coluna de total dos meses contidos

info['TOTAL'] = info.iloc[ : , 4:].sum(axis=1)

# Padronizando as colunas e organiznado

info.rename(columns={'DESCRICAO_UO': 'UO', 'PT': 'PROGRAMA DE TRABALHO',
                      'PT_DESCRICAO': 'PROGRAMA DE TRABALHO DESCRIÇÃO',
                        'NATUREZA6': 'NATUREZA'}, inplace=True)

# Padronizando os meses

meses_padronizados = {
    '1': 'JAN', '2': 'FEV', '3': 'MAR', '4': 'ABR',
    '5': 'MAI', '6': 'JUN', '7': 'JUL', '8': 'AGO',
    '9': 'SET', '10': 'OUT', '11': 'NOV', '12': 'DEZ'
}

info.rename(columns=meses_padronizados, inplace=True)

# Padronizando as naturezas

def formatar_natureza(x):
    x_str = str(x)
    x_mod = x_str[1:]
    x_mod = x_str[:-2]
    return int(x_mod)

info['NATUREZA'] = info['NATUREZA'].apply(formatar_natureza)

# Padronizando os PTs

pt_padronizados = {'10302101550690000': '10.302. 1015. 5069',
                   '6122000420010000': '06.122. 0004. 2001',
                   '20605104335800000': '20.605. 1043. 3580',
                   '8306104352110000': '08.306. 1043. 5211',
                   '12368101237620000': '12.368. 1012. 3762',
                   '12368101237620000': '12.368. 1012. 3762',
                   '6181101738370000': '06.181. 1017. 3837',
                   '14122101751890000': '14.122. 1017. 5189',
                   '14122101751890000': '14.122. 1017. 5189',
                   '14122101751890000': '14.122. 1017. 5189',
                   '14122101751890000': '14.122. 1017. 5189',
                   '14421000451850000': '14.421. 0004. 5185',
                   '14421000451850000': '14.421. 0004. 5185',
                   '14421000451850000': '14.421. 0004. 5185',
                   '6122101952750000': '06.122. 1019. 5275',
                   '6122101952750000': '06.122. 1019. 5275'}

info['PROGRAMA DE TRABALHO'] = info['PROGRAMA DE TRABALHO'].astype(str)
info['PROGRAMA DE TRABALHO'] = info['PROGRAMA DE TRABALHO'].map(pt_padronizados)

# Salvando

info.to_excel(base_arquivo, index=False)

modelo_wb = load_workbook('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/SCRIPTS/Bases empenhado - Layne/Base Acompanhamento - Modelo.xlsx')
modelo_ws = modelo_wb.active

novo_wb = Workbook()
novo_ws = novo_wb.active

# Transferência de dados e formatação do df
for r_idx, row in enumerate(dataframe_to_rows(info, index=False, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        cell = novo_ws.cell(row=r_idx, column=c_idx, value=value)
        modelo_cell = modelo_ws.cell(row=r_idx, column=c_idx)
        if modelo_cell.has_style:
            cell.font = copy(modelo_cell.font)
            cell.border = copy(modelo_cell.border)
            cell.fill = copy(modelo_cell.fill)
            cell.number_format = copy(modelo_cell.number_format)
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

# Ajustar largura das colunas
for col in novo_ws.columns:
    max_length = 0
    column = col[0].column_letter
    for cell in col:
        if cell.value and len(str(cell.value)) > max_length:
            max_length = len(str(cell.value))
    adjusted_width = max_length + 2
    novo_ws.column_dimensions[column].width = adjusted_width

# Definir largura específica para a coluna A e D e centralizar todo o conteúdo
novo_ws.column_dimensions['A'].width = 40
novo_ws.column_dimensions['C'].width = 11
novo_ws.column_dimensions['D'].width = 60

# Ajuste específico para as colunas E até J
for col in ['E', 'F', 'G', 'H', 'I', 'J']:
    novo_ws.column_dimensions[col].width = 20 

# Mesclar células de A18 a D18
novo_ws.merge_cells('A18:D18')

# Formatar célula mesclada A18 a D18 e coluna J conforme a célula modelo
modelo_cell_A18 = modelo_ws['A18']
novo_ws.merge_cells('A18:D18')
merged_cell = novo_ws['A18']
merged_cell.value = "TOTAL"
merged_cell.font = Font(bold=True)
merged_cell.font = copy(modelo_cell_A18.font)
merged_cell.border = copy(modelo_cell_A18.border)
merged_cell.fill = copy(modelo_cell_A18.fill)
merged_cell.alignment = copy(modelo_cell_A18.alignment)

# Inserir fórmula de soma na célula J18 e aplicar formatação do modelo

novo_ws['J18'] = f"=SUM(J2:J17)"
modelo_cell_I1 = modelo_ws['I1']
modelo_cell_I2 = modelo_ws['I2']
novo_ws['J1'].font = copy(modelo_cell_I1.font)
novo_ws['J1'].border = copy(modelo_cell_I1.border)
novo_ws['J1'].fill = copy(modelo_cell_I1.fill)
novo_ws['J1'].number_format = copy(modelo_cell_I1.number_format)
novo_ws['J1'].alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)


for c in range(5, 11):
    for i in range(2, 19):
        novo_ws.cell(row=i, column=c).font = copy(modelo_cell_I2.font)
        novo_ws.cell(row=i, column=c).border = copy(modelo_cell_I2.border)
        novo_ws.cell(row=i, column=c).fill = copy(modelo_cell_I2.fill)
        novo_ws.cell(row=i, column=c).number_format = copy(modelo_cell_I2.number_format)
        novo_ws.cell(row=i, column=c).alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

# Inserir fórmula de soma nas células E18 a I18
for col in range(5, 11):
    cell = novo_ws.cell(row=18, column=col)
    cell.value = f"=SUM({cell.column_letter}2:{cell.column_letter}17)"

novo_wb.save(base_arquivo)

info

,UO,PROGRAMA DE TRABALHO,NATUREZA,PROGRAMA DE TRABALHO DESCRIÇÃO,JAN,FEV,MAR,ABR,MAI,TOTAL
0,FUNDO ESTADUAL DE SAÚDE,10.302. 1015. 5069,3339036,QUALIFICAÇÃO DA ASSISTÊNCIA DE MÉDIA E ALTA CO...,62048990.06,63093518.99,60525791.43,64806749.2,0.0,250475049.68
1,POLÍCIA MILITAR DO ESTADO DE ALAGOAS,06.122. 0004. 2001,3339046,MANUTENÇÃO DAS ATIVIDADES DO ÓRGÃO,0.0,2478693.28,4905910.4,-109565.44,0.0,7275038.24
2,"SECRETARIA DE ESTADO DA AGRICULTURA, PECUÁRIA,...",20.605. 1043. 3580,3339032,FORTALECIMENTO DA AGRICULTURA FAMILIAR,0.0,2059515.64,2005239.07,12973536.67,218182.12,17256473.5
3,SECRETARIA DE ESTADO DA ASSISTÊNCIA E DESENVOL...,08.306. 1043. 5211,3339039,APOIO A POPULAÇÃO EM SITUAÇÃO DE INSEGURANÇA A...,0.0,412648.1,392472.4,400520.52,0.0,1205641.02
4,SECRETARIA DE ESTADO DA EDUCAÇÃO,12.368. 1012. 3762,3339018,IMPLEMENTAÇÃO E FORTALECIMENTO DAS AÇÕES DO PR...,0.0,75000.0,949500.0,991700.0,0.0,2016200.0
5,SECRETARIA DE ESTADO DA EDUCAÇÃO,12.368. 1012. 3762,3449018,IMPLEMENTAÇÃO E FORTALECIMENTO DAS AÇÕES DO PR...,0.0,0.0,0.0,9800550.0,14527150.0,24327700.0
6,SECRETARIA DE ESTADO DA SEGURANÇA PÚBLICA,06.181. 1017. 3837,3339048,IMPLEMENTAÇÃO DO PROGRAMA FORÇA TAREFA,3496902.4,3747579.72,3595932.74,3739772.59,0.0,14580187.45
7,SECRETARIA DE ESTADO DE PREVENÇÃO À VIOLÊNCIA,14.122. 1017. 5189,3339036,MANUTENÇÃO DAS ATIVIDADES DO PROGRAMA RONDA NO...,1115083.62,1364896.39,1314256.32,1212863.91,0.0,5007100.24
8,SECRETARIA DE ESTADO DE PREVENÇÃO À VIOLÊNCIA,14.122. 1017. 5189,3339039,MANUTENÇÃO DAS ATIVIDADES DO PROGRAMA RONDA NO...,50000.0,198105.58,247216.56,238270.28,17556.7,751149.12
9,SECRETARIA DE ESTADO DE PREVENÇÃO À VIOLÊNCIA,14.122. 1017. 5189,3339047,MANUTENÇÃO DAS ATIVIDADES DO PROGRAMA RONDA NO...,0.0,242406.87,266722.12,543154.05,0.0,1052283.04


In [57]:
indices = range(2, 19)
cols = range(5, 11)
from itertools import cycle
colunas_ciclicas = cycle(cols)
pareamentos = zip(colunas_ciclicas, indices)
for i, c in pareamentos:
    print(f'{i}{c}')

(5, 2)
(6, 3)
(7, 4)
(8, 5)
(9, 6)
(10, 7)
(5, 8)
(6, 9)
(7, 10)
(8, 11)
(9, 12)
(10, 13)
(5, 14)
(6, 15)
(7, 16)
(8, 17)
(9, 18)


In [41]:

modelo_wb = load_workbook('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/SCRIPTS/Bases empenhado - Layne/Base Acompanhamento - Modelo.xlsx')
modelo_ws = modelo_wb.active

novo_wb = Workbook()
novo_ws = novo_wb.active

# Transferência de dados e formatação do DataFrame
for r_idx, row in enumerate(dataframe_to_rows(info, index=False, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        cell = novo_ws.cell(row=r_idx, column=c_idx, value=value)
        modelo_cell = modelo_ws.cell(row=r_idx, column=c_idx)
        if modelo_cell.has_style:
            cell.font = copy(modelo_cell.font)
            cell.border = copy(modelo_cell.border)
            cell.fill = copy(modelo_cell.fill)
            cell.number_format = copy(modelo_cell.number_format)
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

# Ajustar largura das colunas
for col in novo_ws.columns:
    max_length = 0
    column = col[0].column_letter
    for cell in col:
        if cell.value and len(str(cell.value)) > max_length:
            max_length = len(str(cell.value))
    adjusted_width = max_length + 2
    novo_ws.column_dimensions[column].width = adjusted_width

# Definir largura específica para a coluna D e centralizar todo o conteúdo
novo_ws.column_dimensions['D'].width = 60
novo_ws.column_dimensions['A'].width = 40

# Ajuste específico para as colunas E até I
for col in ['E', 'F', 'G', 'H', 'I', 'J']:
    novo_ws.column_dimensions[col].width = 20  # Definir uma largura maior, ajuste conforme necessário

# Mesclar células de A18 a D18
novo_ws.merge_cells('A18:D18')

# Formatar célula mesclada A18 a D18 e coluna J conforme a célula modelo
modelo_cell_A18 = modelo_ws['A18']
novo_ws.merge_cells('A18:D18')
merged_cell = novo_ws['A18']
merged_cell.value = "TOTAL"
merged_cell.font = Font(bold=True)  # Aplicar fonte em negrito
merged_cell.font = copy(modelo_cell_A18.font)
merged_cell.border = copy(modelo_cell_A18.border)
merged_cell.fill = copy(modelo_cell_A18.fill)
merged_cell.alignment = copy(modelo_cell_A18.alignment)

# Inserir fórmula de soma na célula J18 e aplicar formatação do modelo
modelo_cell_I1 = modelo_ws['I1']  # Supondo que a formatação de J seja como J1 no modelo
novo_ws['J1'].font = copy(modelo_cell_I1.font)
novo_ws['J1'].border = copy(modelo_cell_I1.border)
novo_ws['J1'].fill = copy(modelo_cell_I1.fill)
novo_ws['J1'].number_format = copy(modelo_cell_I1.number_format)
novo_ws['J1'].alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

modelo_cell_I2 = modelo_ws['I2']  # Supondo que a formatação de J seja como J1 no modelo
novo_ws['J18'] = f"=SUM(J2:J17)"

for i in range(2,19):
    novo_ws[f'J{i}'].font = copy(modelo_cell_I2.font)
    novo_ws[f'J{i}'].border = copy(modelo_cell_I2.border)
    novo_ws[f'J{i}'].fill = copy(modelo_cell_I2.fill)
    novo_ws[f'J{i}'].number_format = copy(modelo_cell_I2.number_format)
    novo_ws[f'J{i}'].alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

# Inserir fórmula de soma nas células E18 a I18
for col in range(5, 11):  # Colunas E a I correspondem aos índices 5 a 9
    cell = novo_ws.cell(row=18, column=col)
    cell.value = f"=SUM({cell.column_letter}2:{cell.column_letter}17)"

novo_wb.save(base_arquivo)


info

,UO,PROGRAMA DE TRABALHO,NATUREZA,PROGRAMA DE TRABALHO DESCRIÇÃO,JAN,FEV,MAR,ABR,MAI,Total
0,FUNDO ESTADUAL DE SAÚDE,10.302. 1015. 5069,3339036.0,QUALIFICAÇÃO DA ASSISTÊNCIA DE MÉDIA E ALTA CO...,62048990.06,63093518.99,60525791.43,64806749.20,0.00,2.504750e+08
1,POLÍCIA MILITAR DO ESTADO DE ALAGOAS,06.122. 0004. 2001,3339046.0,MANUTENÇÃO DAS ATIVIDADES DO ÓRGÃO,0.00,2478693.28,4905910.40,-109565.44,0.00,7.275038e+06
2,"SECRETARIA DE ESTADO DA AGRICULTURA, PECUÁRIA,...",20.605. 1043. 3580,3339032.0,FORTALECIMENTO DA AGRICULTURA FAMILIAR,0.00,2059515.64,2005239.07,12973536.67,218182.12,1.725647e+07
3,SECRETARIA DE ESTADO DA ASSISTÊNCIA E DESENVOL...,08.306. 1043. 5211,3339039.0,APOIO A POPULAÇÃO EM SITUAÇÃO DE INSEGURANÇA A...,0.00,412648.10,392472.40,400520.52,0.00,1.205641e+06
4,SECRETARIA DE ESTADO DA EDUCAÇÃO,12.368. 1012. 3762,3339018.0,IMPLEMENTAÇÃO E FORTALECIMENTO DAS AÇÕES DO PR...,0.00,75000.00,949500.00,991700.00,0.00,2.016200e+06
5,SECRETARIA DE ESTADO DA EDUCAÇÃO,12.368. 1012. 3762,3449018.0,IMPLEMENTAÇÃO E FORTALECIMENTO DAS AÇÕES DO PR...,0.00,0.00,0.00,9800550.00,14527150.00,2.432770e+07
6,SECRETARIA DE ESTADO DA SEGURANÇA PÚBLICA,06.181. 1017. 3837,3339048.0,IMPLEMENTAÇÃO DO PROGRAMA FORÇA TAREFA,3496902.40,3747579.72,3595932.74,3739772.59,0.00,1.458019e+07
7,SECRETARIA DE ESTADO DE PREVENÇÃO À VIOLÊNCIA,14.122. 1017. 5189,3339036.0,MANUTENÇÃO DAS ATIVIDADES DO PROGRAMA RONDA NO...,1115083.62,1364896.39,1314256.32,1212863.91,0.00,5.007100e+06
8,SECRETARIA DE ESTADO DE PREVENÇÃO À VIOLÊNCIA,14.122. 1017. 5189,3339039.0,MANUTENÇÃO DAS ATIVIDADES DO PROGRAMA RONDA NO...,50000.00,198105.58,247216.56,238270.28,17556.70,7.511491e+05
9,SECRETARIA DE ESTADO DE PREVENÇÃO À VIOLÊNCIA,14.122. 1017. 5189,3339047.0,MANUTENÇÃO DAS ATIVIDADES DO PROGRAMA RONDA NO...,0.00,242406.87,266722.12,543154.05,0.00,1.052283e+06


In [33]:
for i in range(2,18):
    print(i)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [16]:

# Carregar o workbook modelo e obter a formatação da célula modelo
modelo_wb = load_workbook('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/SCRIPTS/Bases empenhado - Layne/Base Acompanhamento - Modelo.xlsx')
modelo_ws = modelo_wb.active

novo_wb = Workbook()
novo_ws = novo_wb.active

# Transferência de dados e formatação do DataFrame
for r_idx, row in enumerate(dataframe_to_rows(info, index=False, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        cell = novo_ws.cell(row=r_idx, column=c_idx, value=value)
        modelo_cell = modelo_ws.cell(row=r_idx, column=c_idx)
        if modelo_cell.has_style:
            cell.font = copy(modelo_cell.font)
            cell.border = copy(modelo_cell.border)
            cell.fill = copy(modelo_cell.fill)
            cell.number_format = copy(modelo_cell.number_format)
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

# Ajustar largura específica para a coluna C
novo_ws.column_dimensions['C'].width = 11

# Formatar célula mesclada A18 a D18 e coluna J conforme a célula modelo
modelo_cell_A18 = modelo_ws['A18']
novo_ws.merge_cells('A18:D18')
merged_cell = novo_ws['A18']
merged_cell.value = "TOTAL"
merged_cell.font = copy(modelo_cell_A18.font)
merged_cell.border = copy(modelo_cell_A18.border)
merged_cell.fill = copy(modelo_cell_A18.fill)
merged_cell.alignment = copy(modelo_cell_A18.alignment)

# Inserir fórmula de soma na célula J18 e aplicar formatação do modelo
modelo_cell_J = modelo_ws['J1']  # Supondo que a formatação de J seja como J1 no modelo
novo_ws['J18'] = f"=SUM(J2:J17)"
novo_ws['J18'].font = copy(modelo_cell_J.font)
novo_ws['J18'].border = copy(modelo_cell_J.border)
novo_ws['J18'].fill = copy(modelo_cell_J.fill)
novo_ws['J18'].number_format = copy(modelo_cell_J.number_format)
novo_ws['J18'].alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

novo_wb.save('S:/SOP/003 - GERÊNCIA DE ESTUDOS E PROJEÇÕES/SCRIPTS/Bases empenhado - Layne/Base Acompanhamento - Formatada.xlsx')